2 methods for interpolation: one is to interpolate the observation to the model, the other is to interpolate the model to the observation. I tried doing both.

In [2]:
import xarray as xr
import numpy as np
import xesmf as xe

import fsspec
import os

import datetime
import cftime
import gc

In [3]:
fil1='/scratch/ldoyle4/ufs8/daily/anom/anoms_2year/taux_01_anom.nc'
fil2='/scratch/ldoyle4/ufs8/daily/anom/anoms_2year/tauy_01_anom.nc'
fil3='/scratch/ldoyle4/ufs8/daily/anom/anoms_2year/sst_01_anom.nc'
fil4='/scratch/ldoyle4/ufs8/daily/anom/anoms_2year/ssu_01_anom.nc'
fil5='/scratch/ldoyle4/ufs8/daily/anom/anoms_2year/ssv_01_anom.nc'

In [4]:
ds_taux_1=xr.open_dataset(fil1)
#ds_taux_15=xr.open_dataset(fil2)

ds_tauy_1=xr.open_dataset(fil2)
#ds_tauy_15=xr.open_dataset(fil4)

ds_sst_1=xr.open_dataset(fil3)
#ds_sst_15=xr.open_dataset(fil6)

ds_ssu_1=xr.open_dataset(fil4)
#ds_ssv_15=xr.open_dataset(fil8)

ds_ssv_1=xr.open_dataset(fil5)
#ds_ssu_15=xr.open_dataset(fil10)

In [5]:
ds_sst_1.load()

<xarray.Dataset>
Dimensions:   (lat: 721, lon: 1441, time: 840)
Coordinates:
  * lat       (lat) float32 -90.0 -89.75 -89.5 -89.25 ... 89.25 89.5 89.75 90.0
  * lon       (lon) float32 0.0 0.25 0.5 0.75 1.0 ... 359.2 359.5 359.8 360.0
  * time      (time) object 2014-01-01 00:00:00 ... 2016-01-04 00:00:00
Data variables:
    SST_anom  (time, lat, lon) float32 nan nan nan nan nan ... nan nan nan nan

In [6]:
taux_1 = ds_taux_1.taux_anom.sel(lat=slice(-29,29),lon=slice(124,280))
#taux_15 = ds_taux_15.taux_anom.sel(lat=slice(-29,29),lon=slice(124,280))

tauy_1 = ds_tauy_1.tauy_anom.sel(lat=slice(-29,29),lon=slice(124,280))
#tauy_15 = ds_tauy_15.tauy_anom.sel(lat=slice(-29,29),lon=slice(124,280))

sst_1 = ds_sst_1.SST_anom.sel(lat=slice(-29,29),lon=slice(124,280))
#sst_15 = ds_sst_15.SST_anom.sel(lat=slice(-29,29),lon=slice(124,280))

ssv_1 = ds_ssv_1.SSV_anom.sel(lat=slice(-29,29),lon=slice(124,280))
#ssv_15 = ds_ssv_15.SSV_anom.sel(lat=slice(-29,29),lon=slice(124,280))

ssu_1 = ds_ssu_1.SSU_anom.sel(lat=slice(-29,29),lon=slice(124,280))
#ssu_15 = ds_ssu_15.SSU_anom.sel(lat=slice(-29,29),lon=slice(124,280))

In [7]:
taux_1

<xarray.DataArray 'taux_anom' (time: 840, lat: 233, lon: 625)>
[122325000 values with dtype=float32]
Coordinates:
  * lat      (lat) float32 -29.0 -28.75 -28.5 -28.25 ... 28.25 28.5 28.75 29.0
  * lon      (lon) float32 124.0 124.2 124.5 124.8 ... 279.2 279.5 279.8 280.0
  * time     (time) object 2014-01-01 00:00:00 ... 2016-01-04 00:00:00

Interpolate the EOFs to match the forecasts:

In [9]:
datadir2='/scratch/ldoyle4/windstress/'
uflx_eof = xr.open_dataset(datadir2+'obs_taux_eof_1.nc').taux_eof

In [28]:
uflx_eof.load()

<xarray.DataArray 'taux_eof' (time: 4, lat: 117, lon: 313)>
array([[[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ..., 0.00077314,
         0.0006015 , 0.00079706],
        [       nan,        nan,        nan, ..., 0.00181876,
         0.00038329, 0.00062037],
        ...,
        [0.00243857, 0.0047094 , 0.00415611, ...,        nan,
                nan, 0.00573796],
        [0.00339341, 0.00315317, 0.00273557, ...,        nan,
         0.00668874, 0.00462073],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan]],

       [[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ..., 0.00077314,
         0.0006015 , 0.00079706],
        [       nan,        nan,        nan, ..., 0.00181876,
         0.00038329, 0.00062037],
...
        [0.00243857, 0.0047094 , 0.00415611, ...,        nan,
                nan, 0.00573796],
        [0.00339341, 0.00315317, 0.00273557, ...,        nan,
         0.00668874, 0.00462073],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan]],

       [[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ..., 0.00077314,
         0.0006015 , 0.00079706],
        [       nan,        nan,        nan, ..., 0.00181876,
         0.00038329, 0.00062037],
        ...,
        [0.00243857, 0.0047094 , 0.00415611, ...,        nan,
                nan, 0.00573796],
        [0.00339341, 0.00315317, 0.00273557, ...,        nan,
         0.00668874, 0.00462073],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 1991-01-01 1991-01-02 1991-01-03 1991-01-04
  * lat      (lat) float32 -29.0 -28.5 -28.0 -27.5 -27.0 ... 27.5 28.0 28.5 29.0
  * lon      (lon) float32 124.0 124.5 125.0 125.5 ... 278.5 279.0 279.5 280.0
    num      int64 1
Attributes:
    description:  First 4 EOFs of zonal wind stress

In [11]:
lat = taux_1.lat
lon = taux_1.lon

In [12]:
uflx_eof_int = uflx_eof.interp(lat=lat, lon=lon, method='nearest')

/home/ldoyle4/.conda/envs/clim_model/lib/python3.10/site-packages/xarray/core/missing.py:562: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imin = index.get_loc(minval, method="nearest")
/home/ldoyle4/.conda/envs/clim_model/lib/python3.10/site-packages/xarray/core/missing.py:563: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imax = index.get_loc(maxval, method="nearest")
/home/ldoyle4/.conda/envs/clim_model/lib/python3.10/site-packages/xarray/core/missing.py:562: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imin = index.get_loc(minval, method="nearest")
/home/ldoyle4/.conda/envs/clim_model/lib/python3.10/site-packages/xarray/core/missing.py:563: FutureWarnin

In [13]:
uflx_eof_int

<xarray.DataArray 'taux_eof' (time: 4, lat: 233, lon: 625)>
array([[[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ..., 0.0006015 ,
         0.0006015 , 0.00079706],
        ...,
        [0.00339341, 0.00339341, 0.00315317, ..., 0.00668874,
         0.00668874, 0.00462073],
        [0.00339341, 0.00339341, 0.00315317, ..., 0.00668874,
         0.00668874, 0.00462073],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan]],

       [[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ..., 0.0006015 ,
         0.0006015 , 0.00079706],
...
        [0.00339341, 0.00339341, 0.00315317, ..., 0.00668874,
         0.00668874, 0.00462073],
        [0.00339341, 0.00339341, 0.00315317, ..., 0.00668874,
         0.00668874, 0.00462073],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan]],

       [[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ..., 0.0006015 ,
         0.0006015 , 0.00079706],
        ...,
        [0.00339341, 0.00339341, 0.00315317, ..., 0.00668874,
         0.00668874, 0.00462073],
        [0.00339341, 0.00339341, 0.00315317, ..., 0.00668874,
         0.00668874, 0.00462073],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 1991-01-01 1991-01-02 1991-01-03 1991-01-04
    num      int64 1
  * lat      (lat) float32 -29.0 -28.75 -28.5 -28.25 ... 28.25 28.5 28.75 29.0
  * lon      (lon) float32 124.0 124.2 124.5 124.8 ... 279.2 279.5 279.8 280.0
Attributes:
    description:  First 4 EOFs of zonal wind stress

In [14]:
vflx_eof = xr.open_dataset(datadir2+'obs_tauy_eof.nc').tauy_eof

In [15]:
vflx_eof_int = vflx_eof.interp(lat=lat, lon=lon, method='nearest')

/home/ldoyle4/.conda/envs/clim_model/lib/python3.10/site-packages/xarray/core/missing.py:562: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imin = index.get_loc(minval, method="nearest")
/home/ldoyle4/.conda/envs/clim_model/lib/python3.10/site-packages/xarray/core/missing.py:563: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imax = index.get_loc(maxval, method="nearest")
/home/ldoyle4/.conda/envs/clim_model/lib/python3.10/site-packages/xarray/core/missing.py:562: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imin = index.get_loc(minval, method="nearest")
/home/ldoyle4/.conda/envs/clim_model/lib/python3.10/site-packages/xarray/core/missing.py:563: FutureWarnin

In [16]:
meofs=xr.open_dataset('/scratch/ldoyle4/windstress/final/obs_multivarEOFs.nc').multieof

In [17]:
meof_int = meofs.interp(lon=lon, method='nearest')

/home/ldoyle4/.conda/envs/clim_model/lib/python3.10/site-packages/xarray/core/missing.py:562: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imin = index.get_loc(minval, method="nearest")
/home/ldoyle4/.conda/envs/clim_model/lib/python3.10/site-packages/xarray/core/missing.py:563: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imax = index.get_loc(maxval, method="nearest")


In [18]:
meof_int

<xarray.DataArray 'multieof' (num: 2, var: 2, lon: 625)>
array([[[-0.00139638, -0.00139638, -0.00317488, ..., -0.00331416,
         -0.00331416, -0.00311242],
        [ 0.03027654,  0.03027654,  0.03292723, ..., -0.04609843,
         -0.04609843, -0.04428099]],

       [[-0.00474508, -0.00474508, -0.00655993, ..., -0.00010699,
         -0.00010699, -0.00130082],
        [ 0.00042759,  0.00042759,  0.00095464, ..., -0.00036479,
         -0.00036479, -0.00059596]]])
Coordinates:
  * var      (var) int64 1 2
  * num      (num) int64 1 2
  * lon      (lon) float32 124.0 124.2 124.5 124.8 ... 279.2 279.5 279.8 280.0
Attributes:
    description:  Multivariate EOFs: MJO Wind stress and Sea Surface Temperat...

In [19]:
uflx_eof_int.to_netcdf("/scratch/ldoyle4/mjo_enso_eofs/interp/uflx_eof_interp.nc", 
                         encoding={'lat': {'dtype':'float32','_FillValue': None},
                        'lon': {'dtype':'float32','_FillValue': None},
                        'time': {'dtype':'int32'},
                        'taux_eof':{'dtype':'float32'}})

In [20]:
vflx_eof_int.to_netcdf("/scratch/ldoyle4/mjo_enso_eofs/interp/vflx_eof_interp.nc", 
                         encoding={'lat': {'dtype':'float32','_FillValue': None},
                        'lon': {'dtype':'float32','_FillValue': None},
                        'time': {'dtype':'int32'},
                        'tauy_eof':{'dtype':'float32'}})

In [21]:
meof_int.to_netcdf("/scratch/ldoyle4/mjo_enso_eofs/interp/meof_interp.nc", 
                         encoding={'num': {'dtype':'int32'},
                        'var': {'dtype':'int32'},
                        'lon': {'dtype':'float32','_FillValue': None},
                        'multieof':{'dtype':'float32'}})

In [23]:
taux_1.to_netcdf("/scratch/ldoyle4/sliced_anoms/taux_anom.nc", 
                         encoding={'lat': {'dtype':'float32','_FillValue': None},
                        'lon': {'dtype':'float32','_FillValue': None},
                        'time': {'dtype':'int32'},
                        'taux_anom':{'dtype':'float32'}})

In [24]:
tauy_1.to_netcdf("/scratch/ldoyle4/sliced_anoms/tauy_anom.nc", 
                         encoding={'lat': {'dtype':'float32','_FillValue': None},
                        'lon': {'dtype':'float32','_FillValue': None},
                        'time': {'dtype':'int32'},
                        'tauy_anom':{'dtype':'float32'}})

In [25]:
sst_1.to_netcdf("/scratch/ldoyle4/sliced_anoms/sst_anom.nc", 
                         encoding={'lat': {'dtype':'float32','_FillValue': None},
                        'lon': {'dtype':'float32','_FillValue': None},
                        'time': {'dtype':'int32'},
                        'SST_anom':{'dtype':'float32'}})

In [26]:
ssu_1.to_netcdf("/scratch/ldoyle4/sliced_anoms/ssu_anom.nc", 
                         encoding={'lat': {'dtype':'float32','_FillValue': None},
                        'lon': {'dtype':'float32','_FillValue': None},
                        'time': {'dtype':'int32'},
                        'SSU_anom':{'dtype':'float32'}})

In [27]:
ssv_1.to_netcdf("/scratch/ldoyle4/sliced_anoms/ssv_anom.nc", 
                         encoding={'lat': {'dtype':'float32','_FillValue': None},
                        'lon': {'dtype':'float32','_FillValue': None},
                        'time': {'dtype':'int32'},
                        'SSV_anom':{'dtype':'float32'}})

Interpolate anomalies to match observation grid

In [8]:
obs_grid = xr.open_dataset('/scratch/ldoyle4/windstress/blended.taux.anom.nc').taux
print(obs_grid)

<xarray.DataArray 'taux' (time: 10585, lat: 117, lon: 313)>
[387633285 values with dtype=float64]
Coordinates:
  * time     (time) object 1988-01-01 00:00:00 ... 2016-12-31 00:00:00
  * lat      (lat) float64 -29.0 -28.5 -28.0 -27.5 -27.0 ... 27.5 28.0 28.5 29.0
  * lon      (lon) float64 124.0 124.5 125.0 125.5 ... 278.5 279.0 279.5 280.0
Attributes:
    long_name:  Anomalies from Smooth Daily Climatology
    units:      taux


In [9]:
lat = obs_grid.lat
lon = obs_grid.lon

In [10]:
taux_1_int = taux_1.interp(lat=lat, lon=lon, method='nearest')

/home/ldoyle4/.conda/envs/clim_model/lib/python3.10/site-packages/xarray/core/missing.py:562: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imin = index.get_loc(minval, method="nearest")
/home/ldoyle4/.conda/envs/clim_model/lib/python3.10/site-packages/xarray/core/missing.py:563: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imax = index.get_loc(maxval, method="nearest")
/home/ldoyle4/.conda/envs/clim_model/lib/python3.10/site-packages/xarray/core/missing.py:562: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imin = index.get_loc(minval, method="nearest")
/home/ldoyle4/.conda/envs/clim_model/lib/python3.10/site-packages/xarray/core/missing.py:563: FutureWarnin

In [11]:
taux_1_int

<xarray.DataArray 'taux_anom' (time: 840, lat: 117, lon: 313)>
array([[[        nan,         nan,         nan, ...,  0.0443681 ,
          0.04315883,  0.04178394],
        [        nan,         nan,         nan, ...,  0.04052278,
          0.03901441,  0.03834613],
        [        nan,         nan,         nan, ...,  0.03679171,
          0.0355619 ,  0.03442505],
        ...,
        [-0.01727143, -0.02002306, -0.02410948, ...,         nan,
                 nan,         nan],
        [-0.0228988 , -0.0262286 , -0.03025613, ...,         nan,
                 nan,  0.0071318 ],
        [-0.02852147, -0.03202758, -0.03719902, ...,         nan,
                 nan, -0.0007132 ]],

       [[        nan,         nan,         nan, ...,  0.00951311,
          0.00674772,  0.00412951],
        [        nan,         nan,         nan, ...,  0.00989895,
          0.00721879,  0.00492206],
        [        nan,         nan,         nan, ...,  0.01011144,
          0.00770999,  0.00547311],
...
        [-0.01342573, -0.01128333, -0.00930984, ...,         nan,
                 nan,         nan],
        [-0.01097773, -0.00906713, -0.00631428, ...,         nan,
                 nan, -0.02024702],
        [-0.00721023, -0.00565824, -0.00312126, ...,         nan,
                 nan, -0.02010829]],

       [[        nan,         nan,         nan, ..., -0.01052011,
         -0.01077437, -0.01089424],
        [        nan,         nan,         nan, ..., -0.01106678,
         -0.01186634, -0.01233128],
        [        nan,         nan,         nan, ..., -0.0111532 ,
         -0.01211825, -0.01282641],
        ...,
        [-0.01611747, -0.01604367, -0.01501314, ...,         nan,
                 nan,         nan],
        [-0.02102279, -0.02079185, -0.01934772, ...,         nan,
                 nan,  0.01256027],
        [-0.02653468, -0.02466339, -0.02293173, ...,         nan,
                 nan,  0.00932581]]], dtype=float32)
Coordinates:
  * time     (time) object 2014-01-01 00:00:00 ... 2016-01-04 00:00:00
  * lat      (lat) float64 -29.0 -28.5 -28.0 -27.5 -27.0 ... 27.5 28.0 28.5 29.0
  * lon      (lon) float64 124.0 124.5 125.0 125.5 ... 278.5 279.0 279.5 280.0

In [12]:
tauy_1_int = tauy_1.interp(lat=lat, lon=lon, method='nearest')

/home/ldoyle4/.conda/envs/clim_model/lib/python3.10/site-packages/xarray/core/missing.py:562: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imin = index.get_loc(minval, method="nearest")
/home/ldoyle4/.conda/envs/clim_model/lib/python3.10/site-packages/xarray/core/missing.py:563: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imax = index.get_loc(maxval, method="nearest")
/home/ldoyle4/.conda/envs/clim_model/lib/python3.10/site-packages/xarray/core/missing.py:562: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imin = index.get_loc(minval, method="nearest")
/home/ldoyle4/.conda/envs/clim_model/lib/python3.10/site-packages/xarray/core/missing.py:563: FutureWarnin

In [13]:
sst_1_int = sst_1.interp(lat=lat, lon=lon, method='nearest')

/home/ldoyle4/.conda/envs/clim_model/lib/python3.10/site-packages/xarray/core/missing.py:562: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imin = index.get_loc(minval, method="nearest")
/home/ldoyle4/.conda/envs/clim_model/lib/python3.10/site-packages/xarray/core/missing.py:563: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imax = index.get_loc(maxval, method="nearest")
/home/ldoyle4/.conda/envs/clim_model/lib/python3.10/site-packages/xarray/core/missing.py:562: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imin = index.get_loc(minval, method="nearest")
/home/ldoyle4/.conda/envs/clim_model/lib/python3.10/site-packages/xarray/core/missing.py:563: FutureWarnin

In [14]:
ssu_1_int = ssu_1.interp(lat=lat, lon=lon, method='nearest')

/home/ldoyle4/.conda/envs/clim_model/lib/python3.10/site-packages/xarray/core/missing.py:562: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imin = index.get_loc(minval, method="nearest")
/home/ldoyle4/.conda/envs/clim_model/lib/python3.10/site-packages/xarray/core/missing.py:563: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imax = index.get_loc(maxval, method="nearest")
/home/ldoyle4/.conda/envs/clim_model/lib/python3.10/site-packages/xarray/core/missing.py:562: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imin = index.get_loc(minval, method="nearest")
/home/ldoyle4/.conda/envs/clim_model/lib/python3.10/site-packages/xarray/core/missing.py:563: FutureWarnin

In [15]:
ssv_1_int = ssv_1.interp(lat=lat, lon=lon, method='nearest')

/home/ldoyle4/.conda/envs/clim_model/lib/python3.10/site-packages/xarray/core/missing.py:562: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imin = index.get_loc(minval, method="nearest")
/home/ldoyle4/.conda/envs/clim_model/lib/python3.10/site-packages/xarray/core/missing.py:563: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imax = index.get_loc(maxval, method="nearest")
/home/ldoyle4/.conda/envs/clim_model/lib/python3.10/site-packages/xarray/core/missing.py:562: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imin = index.get_loc(minval, method="nearest")
/home/ldoyle4/.conda/envs/clim_model/lib/python3.10/site-packages/xarray/core/missing.py:563: FutureWarnin

In [16]:
sst_2014_01 = sst_1_int[0:35,:,:]
sst_2014_02 = sst_1_int[35:70,:,:]
sst_2014_03 = sst_1_int[70:105,:,:]
sst_2014_04 = sst_1_int[105:140,:,:]
sst_2014_05 = sst_1_int[140:175,:,:]
sst_2014_06 = sst_1_int[175:210,:,:]
sst_2014_07 = sst_1_int[210:245,:,:]
sst_2014_08 = sst_1_int[245:280,:,:]
sst_2014_09 = sst_1_int[280:315,:,:]
sst_2014_10 = sst_1_int[315:350,:,:]
sst_2014_11 = sst_1_int[350:385,:,:]
sst_2014_12 = sst_1_int[385:420,:,:]
sst_2015_01 = sst_1_int[420:455,:,:]
sst_2015_02 = sst_1_int[455:490,:,:]
sst_2015_03 = sst_1_int[490:525,:,:]
sst_2015_04 = sst_1_int[525:560,:,:]
sst_2015_05 = sst_1_int[560:595,:,:]
sst_2015_06 = sst_1_int[595:630,:,:]
sst_2015_07 = sst_1_int[630:665,:,:]
sst_2015_08 = sst_1_int[665:700,:,:]
sst_2015_09 = sst_1_int[700:735,:,:]
sst_2015_10 = sst_1_int[735:770,:,:]
sst_2015_11 = sst_1_int[770:805,:,:]
sst_2015_12 = sst_1_int[805:840,:,:]

In [17]:
sst_2014_01

<xarray.DataArray 'SST_anom' (time: 35, lat: 117, lon: 313)>
array([[[        nan,         nan,         nan, ...,  0.44244385,
          0.43912888,  0.43624878],
        [        nan,         nan,         nan, ...,  0.35570335,
          0.3837242 ,  0.26413536],
        [        nan,         nan,         nan, ...,  0.29744148,
          0.27874756,  0.22744751],
        ...,
        [ 0.63941956,  0.65491295,  0.53863335, ...,         nan,
                 nan, -0.3400612 ],
        [ 0.19686127,  0.0397644 ,  0.26763535, ...,         nan,
                 nan, -0.5005779 ],
        [-0.03420639, -0.13879776, -0.17536926, ...,         nan,
                 nan, -0.38834763]],

       [[        nan,         nan,         nan, ...,  0.5748348 ,
          0.5779762 ,  0.61356926],
        [        nan,         nan,         nan, ...,  0.52760696,
          0.5459461 ,  0.45092773],
        [        nan,         nan,         nan, ...,  0.4515648 ,
          0.46780777,  0.42713547],
...
        [-0.28842926,  0.29989815,  0.4485569 , ...,         nan,
                 nan,  0.4959736 ],
        [ 0.09711075,  0.6175022 ,  0.6456547 , ...,         nan,
                 nan, -0.8497467 ],
        [ 0.24276924,  0.2443409 ,  0.30825615, ...,         nan,
                 nan,  1.006628  ]],

       [[        nan,         nan,         nan, ...,  0.20477104,
          0.20521736,  0.16894722],
        [        nan,         nan,         nan, ...,  0.24077988,
          0.28309822,  0.27274704],
        [        nan,         nan,         nan, ...,  0.21098709,
          0.1816063 ,  0.20272255],
        ...,
        [-0.1999588 ,  0.3170185 ,  0.6971836 , ...,         nan,
                 nan,  0.05917358],
        [ 0.1552906 ,  0.63523674,  0.6058979 , ...,         nan,
                 nan, -1.1872272 ],
        [ 0.2903452 ,  0.2433796 ,  0.28443718, ...,         nan,
                 nan,  1.0348797 ]]], dtype=float32)
Coordinates:
  * time     (time) object 2014-01-01 00:00:00 ... 2014-02-04 00:00:00
  * lat      (lat) float64 -29.0 -28.5 -28.0 -27.5 -27.0 ... 27.5 28.0 28.5 29.0
  * lon      (lon) float64 124.0 124.5 125.0 125.5 ... 278.5 279.0 279.5 280.0

In [18]:
ssu_2014_01 = ssu_1_int[0:35,:,:]
ssu_2014_02 = ssu_1_int[35:70,:,:]
ssu_2014_03 = ssu_1_int[70:105,:,:]
ssu_2014_04 = ssu_1_int[105:140,:,:]
ssu_2014_05 = ssu_1_int[140:175,:,:]
ssu_2014_06 = ssu_1_int[175:210,:,:]
ssu_2014_07 = ssu_1_int[210:245,:,:]
ssu_2014_08 = ssu_1_int[245:280,:,:]
ssu_2014_09 = ssu_1_int[280:315,:,:]
ssu_2014_10 = ssu_1_int[315:350,:,:]
ssu_2014_11 = ssu_1_int[350:385,:,:]
ssu_2014_12 = ssu_1_int[385:420,:,:]
ssu_2015_01 = ssu_1_int[420:455,:,:]
ssu_2015_02 = ssu_1_int[455:490,:,:]
ssu_2015_03 = ssu_1_int[490:525,:,:]
ssu_2015_04 = ssu_1_int[525:560,:,:]
ssu_2015_05 = ssu_1_int[560:595,:,:]
ssu_2015_06 = ssu_1_int[595:630,:,:]
ssu_2015_07 = ssu_1_int[630:665,:,:]
ssu_2015_08 = ssu_1_int[665:700,:,:]
ssu_2015_09 = ssu_1_int[700:735,:,:]
ssu_2015_10 = ssu_1_int[735:770,:,:]
ssu_2015_11 = ssu_1_int[770:805,:,:]
ssu_2015_12 = ssu_1_int[805:840,:,:]

In [19]:
ssv_2014_01 = ssv_1_int[0:35,:,:]
ssv_2014_02 = ssv_1_int[35:70,:,:]
ssv_2014_03 = ssv_1_int[70:105,:,:]
ssv_2014_04 = ssv_1_int[105:140,:,:]
ssv_2014_05 = ssv_1_int[140:175,:,:]
ssv_2014_06 = ssv_1_int[175:210,:,:]
ssv_2014_07 = ssv_1_int[210:245,:,:]
ssv_2014_08 = ssv_1_int[245:280,:,:]
ssv_2014_09 = ssv_1_int[280:315,:,:]
ssv_2014_10 = ssv_1_int[315:350,:,:]
ssv_2014_11 = ssv_1_int[350:385,:,:]
ssv_2014_12 = ssv_1_int[385:420,:,:]
ssv_2015_01 = ssv_1_int[420:455,:,:]
ssv_2015_02 = ssv_1_int[455:490,:,:]
ssv_2015_03 = ssv_1_int[490:525,:,:]
ssv_2015_04 = ssv_1_int[525:560,:,:]
ssv_2015_05 = ssv_1_int[560:595,:,:]
ssv_2015_06 = ssv_1_int[595:630,:,:]
ssv_2015_07 = ssv_1_int[630:665,:,:]
ssv_2015_08 = ssv_1_int[665:700,:,:]
ssv_2015_09 = ssv_1_int[700:735,:,:]
ssv_2015_10 = ssv_1_int[735:770,:,:]
ssv_2015_11 = ssv_1_int[770:805,:,:]
ssv_2015_12 = ssv_1_int[805:840,:,:]

In [20]:
taux_2014_01 = taux_1_int[0:35,:,:]
taux_2014_02 = taux_1_int[35:70,:,:]
taux_2014_03 = taux_1_int[70:105,:,:]
taux_2014_04 = taux_1_int[105:140,:,:]
taux_2014_05 = taux_1_int[140:175,:,:]
taux_2014_06 = taux_1_int[175:210,:,:]
taux_2014_07 = taux_1_int[210:245,:,:]
taux_2014_08 = taux_1_int[245:280,:,:]
taux_2014_09 = taux_1_int[280:315,:,:]
taux_2014_10 = taux_1_int[315:350,:,:]
taux_2014_11 = taux_1_int[350:385,:,:]
taux_2014_12 = taux_1_int[385:420,:,:]
taux_2015_01 = taux_1_int[420:455,:,:]
taux_2015_02 = taux_1_int[455:490,:,:]
taux_2015_03 = taux_1_int[490:525,:,:]
taux_2015_04 = taux_1_int[525:560,:,:]
taux_2015_05 = taux_1_int[560:595,:,:]
taux_2015_06 = taux_1_int[595:630,:,:]
taux_2015_07 = taux_1_int[630:665,:,:]
taux_2015_08 = taux_1_int[665:700,:,:]
taux_2015_09 = taux_1_int[700:735,:,:]
taux_2015_10 = taux_1_int[735:770,:,:]
taux_2015_11 = taux_1_int[770:805,:,:]
taux_2015_12 = taux_1_int[805:840,:,:]

In [21]:
tauy_2014_01 = tauy_1_int[0:35,:,:]
tauy_2014_02 = tauy_1_int[35:70,:,:]
tauy_2014_03 = tauy_1_int[70:105,:,:]
tauy_2014_04 = tauy_1_int[105:140,:,:]
tauy_2014_05 = tauy_1_int[140:175,:,:]
tauy_2014_06 = tauy_1_int[175:210,:,:]
tauy_2014_07 = tauy_1_int[210:245,:,:]
tauy_2014_08 = tauy_1_int[245:280,:,:]
tauy_2014_09 = tauy_1_int[280:315,:,:]
tauy_2014_10 = tauy_1_int[315:350,:,:]
tauy_2014_11 = tauy_1_int[350:385,:,:]
tauy_2014_12 = tauy_1_int[385:420,:,:]
tauy_2015_01 = tauy_1_int[420:455,:,:]
tauy_2015_02 = tauy_1_int[455:490,:,:]
tauy_2015_03 = tauy_1_int[490:525,:,:]
tauy_2015_04 = tauy_1_int[525:560,:,:]
tauy_2015_05 = tauy_1_int[560:595,:,:]
tauy_2015_06 = tauy_1_int[595:630,:,:]
tauy_2015_07 = tauy_1_int[630:665,:,:]
tauy_2015_08 = tauy_1_int[665:700,:,:]
tauy_2015_09 = tauy_1_int[700:735,:,:]
tauy_2015_10 = tauy_1_int[735:770,:,:]
tauy_2015_11 = tauy_1_int[770:805,:,:]
tauy_2015_12 = tauy_1_int[805:840,:,:]

In [22]:
sst_list = [sst_2014_01, sst_2014_02, sst_2014_03, sst_2014_04, sst_2014_05, sst_2014_06, sst_2014_07,
            sst_2014_08, sst_2014_09, sst_2014_10, sst_2014_11, sst_2014_12, sst_2015_01, sst_2015_02,
            sst_2015_03, sst_2015_04, sst_2015_05, sst_2015_06, sst_2015_07, sst_2015_08, sst_2015_09,
            sst_2015_10, sst_2015_11, sst_2015_12]

In [23]:
ssu_list = [ssu_2014_01, ssu_2014_02, ssu_2014_03, ssu_2014_04, ssu_2014_05, ssu_2014_06, ssu_2014_07,
            ssu_2014_08, ssu_2014_09, ssu_2014_10, ssu_2014_11, ssu_2014_12, ssu_2015_01, ssu_2015_02,
            ssu_2015_03, ssu_2015_04, ssu_2015_05, ssu_2015_06, ssu_2015_07, ssu_2015_08, ssu_2015_09,
            ssu_2015_10, ssu_2015_11, ssu_2015_12]

In [24]:
ssv_list = [ssv_2014_01, ssv_2014_02, ssv_2014_03, ssv_2014_04, ssv_2014_05, ssv_2014_06, ssv_2014_07,
            ssv_2014_08, ssv_2014_09, ssv_2014_10, ssv_2014_11, ssv_2014_12, ssv_2015_01, ssv_2015_02,
            ssv_2015_03, ssv_2015_04, ssv_2015_05, ssv_2015_06, ssv_2015_07, ssv_2015_08, ssv_2015_09,
            ssv_2015_10, ssv_2015_11, ssv_2015_12]

In [25]:
taux_list = [taux_2014_01, taux_2014_02, taux_2014_03, taux_2014_04, taux_2014_05, taux_2014_06, taux_2014_07,
            taux_2014_08, taux_2014_09, taux_2014_10, taux_2014_11, taux_2014_12, taux_2015_01, taux_2015_02,
            taux_2015_03, taux_2015_04, taux_2015_05, taux_2015_06, taux_2015_07, taux_2015_08, taux_2015_09,
            taux_2015_10, taux_2015_11, taux_2015_12]

In [26]:
tauy_list = [tauy_2014_01, tauy_2014_02, tauy_2014_03, tauy_2014_04, tauy_2014_05, tauy_2014_06, tauy_2014_07,
            tauy_2014_08, tauy_2014_09, tauy_2014_10, tauy_2014_11, tauy_2014_12, tauy_2015_01, tauy_2015_02,
            tauy_2015_03, tauy_2015_04, tauy_2015_05, tauy_2015_06, tauy_2015_07, tauy_2015_08, tauy_2015_09,
            tauy_2015_10, tauy_2015_11, tauy_2015_12]

In [27]:
years = ['2014','2015']

months_num = np.arange(1,13,1)
months = []
for month in months_num: months.append('{:02d}'.format(month))
print(months)

day_init = ['01'] # or ['01']

['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']


In [28]:
date_for_file = [f'{year}{month}{day}' for year in years for month in months for day in day_init]
print(date_for_file)

['20140101', '20140201', '20140301', '20140401', '20140501', '20140601', '20140701', '20140801', '20140901', '20141001', '20141101', '20141201', '20150101', '20150201', '20150301', '20150401', '20150501', '20150601', '20150701', '20150801', '20150901', '20151001', '20151101', '20151201']


In [47]:
def save_as(path, dataset, date_range, encoding):
    """
    Given a path, the dataset, the date range, and the encoding: will write out the data as a netcdf and save to the path given.
    
    Make sure to change the variable!
    
    The structure of the function return should have the format:
    return(dataset.to_netcdf((path)+(variable: taux, tauy, SST, SSU, SSV)+"_"+(date_range)+".nc", encoding=encoding))
    """
    
    return(dataset.to_netcdf((path)+"tauy_"+(date_range)+".nc", encoding=encoding))


In [48]:
for i in range(len(date_for_file)):
    save_as(path= "/scratch/ldoyle4/new_anoms/slice_interp_split/",
            dataset = tauy_list[i],
            date_range = date_for_file[i],
            encoding = {'lat': {'dtype':'float32','_FillValue': None},
                'lon': {'dtype':'float32','_FillValue': None},
                'time': {'dtype':'int32'},
                'tauy_anom':{'dtype':'float32'}
                })